In [12]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv("./dataset.csv", encoding="ISO-8859-1")
# initial conclusions about data
df.info()
# no missing values
# I'll need to convert weight, ram, fromt cam, back cam, battery capactiy, and screensize into numeric types
# company name, model name, and processor will all need categorical feature preprocessing, probably one hot label encoding

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Company Name               930 non-null    object
 1   Model Name                 930 non-null    object
 2   Mobile Weight              930 non-null    object
 3   RAM                        930 non-null    object
 4   Front Camera               930 non-null    object
 5   Back Camera                930 non-null    object
 6   Processor                  930 non-null    object
 7   Battery Capacity           930 non-null    object
 8   Screen Size                930 non-null    object
 9   Launched Price (Pakistan)  930 non-null    object
 10  Launched Price (India)     930 non-null    object
 11  Launched Price (China)     930 non-null    object
 12  Launched Price (USA)       930 non-null    object
 13  Launched Price (Dubai)     930 non-null    object
 14  Launched Y

In [13]:
# PREPROCESSING

# TODO: keep a copy of un normalized dataframe for plotting conclusions, etc

# numerical feature conversion and normalization for weight, ram, front cam, back cam, battery, screen size, and usd price
# drop unused columns
df = df.drop(['Launched Price (Pakistan)', 'Launched Price (India)', 'Launched Price (China)', 'Launched Price (Dubai)'], axis=1)
# there are some decmials so cast weight as float
df['Mobile Weight'] = df['Mobile Weight'].astype(str).str.replace('g', '').astype(float)
# normalize
#df['Mobile Weight'] = (df['Mobile Weight'] - np.mean(df['Mobile Weight']))/np.std(df['Mobile Weight'])

# there are some RAM values like 8GB / 12GB (like the Huawei P60 series). we are going to use the first value as that is what the base model uses which is the price listed.
# there are also some decimal values, so we'll cast to float
df['RAM'] = df['RAM'].astype(str).str.split('/').str[0].str.replace('GB', '').astype(float)
# normalize
#df['RAM'] = (df['RAM'] - np.mean(df['RAM']))/np.std(df['RAM'])

# there are some front camera values formatted like 12MP / 4K or 12MP + 12MP so we are just using the first values for now. there is 2 values that starts with 'Dual' which we will test for manually and simply remove the characters
for idx, val in df['Front Camera'].items():
    if val[0] == 'D':
        val = val[5:]
    t = float(val.split('MP')[0])
    df.at[idx, 'Front Camera'] = t
# normalize
#df['Front Camera'] = (df['Front Camera'] - np.mean(df['Front Camera']))/np.std(df['Front Camera'])

# do the same for back camera
for idx, val in df['Back Camera'].items():
    if val[0] == 'D':
        val = val[5:]
    t = float(val.split('MP')[0])
    df.at[idx, 'Back Camera'] = t
# normalize
# df['Back Camera'] = (df['Back Camera'] - np.mean(df['Back Camera']))/np.std(df['Back Camera'])

# all values in front camera/back camera are floats now. this next line just tells pandas to recheck the type essentially
df = df.infer_objects()

# for battery capacity we just need to remove the mAh, remove the comma, and cast to int
df['Battery Capacity'] = df['Battery Capacity'].astype(str).str.replace('mAh', '').str.replace(',', '').astype(int)
# normalize
#df['Battery Capacity'] = (df['Battery Capacity'] - np.mean(df['Battery Capacity']))/np.std(df['Battery Capacity'])

# for screen size we will split on the first space and cast to float. there are some with external screens (the samsung folds and flips for example) but for now we will just use the first value
df['Screen Size'] = df['Screen Size'].astype(str).str.split(r'\s+').str[0].astype(float)
# normalize
#df['Screen Size'] = (df['Screen Size'] - np.mean(df['Screen Size']))/np.std(df['Screen Size'])

# for price, just remove the 'USD ' and the comma
df['Launched Price (USA)'] = df['Launched Price (USA)'].astype(str).str.replace('USD ', '').str.replace(',', '').astype(float)
# normalize
#df['Launched Price (USA)'] = (df['Launched Price (USA)'] - np.mean(df['Launched Price (USA)']))/np.std(df['Launched Price (USA)'])

# normalize year 
#df['Launched Year'] = (df['Launched Year'] - np.mean(df['Launched Year']))/np.std(df['Launched Year'])


# categorical feature conversion for company, model, and processor. keep a copy of each encoder to unencode values for conclusions
companyLE = LabelEncoder()
df['Company Name'] = companyLE.fit_transform(df['Company Name'])

modelLE = LabelEncoder()
df['Model Name'] = modelLE.fit_transform(df['Model Name'])

processorLE = LabelEncoder()
df['Processor'] = processorLE.fit_transform(df['Processor'])

#df.info()
#df.head()
df.corr()

,Company Name,Model Name,Mobile Weight,RAM,Front Camera,Back Camera,Processor,Battery Capacity,Screen Size,Launched Price (USA),Launched Year
Company Name,1.000000,-0.145911,-0.076564,0.126444,0.169474,0.169655,0.245158,0.079270,-0.059101,-0.083653,0.124861
Model Name,-0.145911,1.000000,0.090522,-0.116841,-0.052529,-0.196505,-0.282567,-0.087496,0.038430,0.079113,-0.231134
Mobile Weight,-0.076564,0.090522,1.000000,-0.007427,-0.283264,-0.308513,-0.000904,0.847731,0.975893,0.104919,0.066349
RAM,0.126444,-0.116841,-0.007427,1.000000,0.460093,0.439636,0.135241,0.139213,0.046963,0.108546,0.380281
Front Camera,0.169474,-0.052529,-0.283264,0.460093,1.000000,0.449325,0.056516,-0.159587,-0.224311,-0.003924,0.166053
Back Camera,0.169655,-0.196505,-0.308513,0.439636,0.449325,1.000000,0.046943,-0.105986,-0.263279,-0.014025,0.318188
Processor,0.245158,-0.282567,-0.000904,0.135241,0.056516,0.046943,1.000000,0.083659,0.044587,0.013539,0.061841
Battery Capacity,0.079270,-0.087496,0.847731,0.139213,-0.159587,-0.105986,0.083659,1.000000,0.879453,0.063929,0.351973
Screen Size,-0.059101,0.038430,0.975893,0.046963,-0.224311,-0.263279,0.044587,0.879453,1.000000,0.095838,0.139199
Launched Price (USA),-0.083653,0.079113,0.104919,0.108546,-0.003924,-0.014025,0.013539,0.063929,0.095838,1.000000,0.013456


In [14]:
# see above for TODO on keeping unnormalized array
df.median(numeric_only=True)
#df.mean(numeric_only=True)

Company Name               9.00
Model Name               449.50
Mobile Weight            194.00
RAM                        8.00
Front Camera              16.00
Back Camera               50.00
Processor                 98.50
Battery Capacity        5000.00
Screen Size                6.67
Launched Price (USA)     449.00
Launched Year           2023.00
dtype: float64